# Gravitational Wave Signal Analysis

Complete tutorial on analyzing gravitational wave signals from binary black hole mergers.

## Overview

Gravitational waves are ripples in spacetime caused by accelerating massive objects. This notebook demonstrates:

- **Signal Generation**: Simulated binary black hole merger waveforms
- **Noise Modeling**: Detector noise characteristics (LIGO-like)
- **Matched Filtering**: Optimal signal detection method
- **Parameter Estimation**: Mass, distance, coalescence time
- **Visualization**: Time series, spectrograms, strain data

## Binary Black Hole System

- **Mass 1**: 36 solar masses
- **Mass 2**: 29 solar masses  
- **Distance**: 410 Mpc
- **Detector**: LIGO Hanford (H1)

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import fft, signal
from scipy.signal import butter, filtfilt

warnings.filterwarnings("ignore")

plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("Set2")
%matplotlib inline

print("✓ Setup complete")

## 1. Generate Gravitational Wave Signal

In [ ]:
# System parameters
m1 = 36.0  # Solar masses
m2 = 29.0  # Solar masses
distance = 410  # Mpc
sample_rate = 4096  # Hz
duration = 4  # seconds

# Time array
t = np.linspace(0, duration, duration * sample_rate)

# Chirp mass
M_total = m1 + m2
M_chirp = (m1 * m2) ** (3 / 5) / M_total ** (1 / 5)

print("System Parameters:")
print(f"  Mass 1: {m1} M☉")
print(f"  Mass 2: {m2} M☉")
print(f"  Total Mass: {M_total:.1f} M☉")
print(f"  Chirp Mass: {M_chirp:.1f} M☉")
print(f"  Distance: {distance} Mpc")
print(f"  Sample Rate: {sample_rate} Hz")

In [ ]:
def generate_chirp(t, m1, m2, f_low=20, f_high=250):
    """
    Generate simplified inspiral chirp waveform.
    Uses post-Newtonian approximation.
    """
    # Constants (in geometric units)
    G = 6.67430e-11  # m^3 kg^-1 s^-2
    c = 299792458  # m/s
    M_sun = 1.989e30  # kg

    # Convert masses to SI
    m1_kg = m1 * M_sun
    m2_kg = m2 * M_sun
    M = m1_kg + m2_kg
    eta = (m1_kg * m2_kg) / M**2  # Symmetric mass ratio
    M_chirp_kg = M * eta ** (3 / 5)

    # Time to coalescence (simplified)
    t_coal = 2.0  # Place merger at t=2s
    tau = t_coal - t
    tau = np.maximum(tau, 1e-3)  # Avoid singularity

    # Frequency evolution (simplified post-Newtonian)
    f = (1 / (8 * np.pi)) * (5 / (256 * tau)) ** (3 / 8) * (G * M_chirp_kg / c**3) ** (-5 / 8)

    # Clip frequency range
    f = np.clip(f, f_low, f_high)

    # Phase evolution
    phase = 2 * np.pi * np.cumsum(f) / sample_rate

    # Amplitude (simplified, includes 1/distance)
    # Scale to realistic strain amplitude
    A0 = 1e-21 * (100 / distance)  # Rough scaling
    amplitude = A0 * tau ** (-1 / 4)

    # Waveform
    h = amplitude * np.cos(phase)

    return h, f


# Generate signal
h_signal, freq_evolution = generate_chirp(t, m1, m2)

print("\nGenerated chirp signal:")
print(f"  Peak strain: {np.max(np.abs(h_signal)):.2e}")
print(f"  Initial frequency: {freq_evolution[0]:.1f} Hz")
print(f"  Final frequency: {freq_evolution[-1]:.1f} Hz")

## 2. Add Detector Noise

In [ ]:
def generate_detector_noise(t, sample_rate, noise_level=1e-22):
    """
    Generate colored noise approximating LIGO detector noise.
    """
    # White noise
    noise = np.random.normal(0, noise_level, len(t))

    # Color the noise (1/f^2 at low frequencies)
    # Apply bandpass filter to approximate LIGO sensitivity
    nyquist = sample_rate / 2
    low = 20 / nyquist
    high = 500 / nyquist
    b, a = butter(4, [low, high], btype="band")
    noise_colored = filtfilt(b, a, noise)

    # Scale to appropriate level
    noise_colored *= noise_level / np.std(noise_colored)

    return noise_colored


# Generate noise
noise = generate_detector_noise(t, sample_rate, noise_level=5e-22)

# Add noise to signal
data = h_signal + noise

print("Detector noise:")
print(f"  RMS noise: {np.std(noise):.2e}")
print(f"  Signal-to-noise ratio: {np.max(np.abs(h_signal)) / np.std(noise):.1f}")

## 3. Visualize Time Series

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# Signal only
axes[0].plot(t, h_signal, linewidth=0.8)
axes[0].set_title("Pure Gravitational Wave Signal (No Noise)", fontsize=12, fontweight="bold")
axes[0].set_xlabel("Time (s)")
axes[0].set_ylabel("Strain")
axes[0].grid(True, alpha=0.3)
axes[0].ticklabel_format(style="scientific", axis="y", scilimits=(0, 0))

# Noise only
axes[1].plot(t, noise, linewidth=0.5, alpha=0.7, color="orange")
axes[1].set_title("Detector Noise", fontsize=12, fontweight="bold")
axes[1].set_xlabel("Time (s)")
axes[1].set_ylabel("Strain")
axes[1].grid(True, alpha=0.3)
axes[1].ticklabel_format(style="scientific", axis="y", scilimits=(0, 0))

# Signal + Noise
axes[2].plot(t, data, linewidth=0.5, alpha=0.8, color="green")
axes[2].set_title("Observed Data (Signal + Noise)", fontsize=12, fontweight="bold")
axes[2].set_xlabel("Time (s)")
axes[2].set_ylabel("Strain")
axes[2].grid(True, alpha=0.3)
axes[2].ticklabel_format(style="scientific", axis="y", scilimits=(0, 0))

plt.tight_layout()
plt.show()

## 4. Frequency Domain Analysis

In [ ]:
# Compute FFT
fft_data = fft.fft(data)
fft_signal = fft.fft(h_signal)
freqs = fft.fftfreq(len(data), 1 / sample_rate)

# Positive frequencies only
pos_mask = freqs > 0
freqs_pos = freqs[pos_mask]
fft_data_pos = np.abs(fft_data[pos_mask])
fft_signal_pos = np.abs(fft_signal[pos_mask])

# Plot
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Power spectral density
axes[0].loglog(freqs_pos, fft_signal_pos, label="Signal", linewidth=2)
axes[0].loglog(freqs_pos, fft_data_pos, label="Signal + Noise", alpha=0.7, linewidth=1)
axes[0].set_xlim(10, 1000)
axes[0].set_title("Frequency Domain", fontsize=12, fontweight="bold")
axes[0].set_xlabel("Frequency (Hz)")
axes[0].set_ylabel("Amplitude")
axes[0].legend()
axes[0].grid(True, alpha=0.3, which="both")

# Spectrogram
f_spec, t_spec, Sxx = signal.spectrogram(data, sample_rate, nperseg=512)
im = axes[1].pcolormesh(t_spec, f_spec, 10 * np.log10(Sxx), shading="gouraud", cmap="viridis")
axes[1].set_ylim(20, 500)
axes[1].set_title("Spectrogram (Time-Frequency)", fontsize=12, fontweight="bold")
axes[1].set_xlabel("Time (s)")
axes[1].set_ylabel("Frequency (Hz)")
plt.colorbar(im, ax=axes[1], label="Power (dB)")

plt.tight_layout()
plt.show()

print("Chirp visible in spectrogram: frequency increases over time!")

## 5. Matched Filtering

In [ ]:
def matched_filter(data, template):
    """
    Compute matched filter SNR time series.
    """
    # Normalize template
    template_norm = template / np.sqrt(np.sum(template**2))

    # Cross-correlation
    snr = signal.correlate(data, template_norm, mode="same")

    # Normalize by noise
    # Estimate noise std from beginning (before signal)
    noise_std = np.std(data[: len(data) // 4])
    snr = snr / noise_std

    return snr


# Use pure signal as template
template = h_signal

# Compute matched filter SNR
snr = matched_filter(data, template)

# Find peak
peak_idx = np.argmax(np.abs(snr))
peak_snr = snr[peak_idx]
peak_time = t[peak_idx]

print("Matched Filter Results:")
print(f"  Peak SNR: {np.abs(peak_snr):.1f}")
print(f"  Detection time: {peak_time:.3f} s")
print(f"  Threshold (SNR > 8): {'DETECTED!' if np.abs(peak_snr) > 8 else 'Not detected'}")

In [ ]:
# Visualize matched filter output
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Data
axes[0].plot(t, data, linewidth=0.5, alpha=0.7)
axes[0].axvline(
    peak_time, color="red", linestyle="--", linewidth=2, label=f"Detection at t={peak_time:.3f}s"
)
axes[0].set_title("Detector Data", fontsize=12, fontweight="bold")
axes[0].set_xlabel("Time (s)")
axes[0].set_ylabel("Strain")
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].ticklabel_format(style="scientific", axis="y", scilimits=(0, 0))

# Matched filter SNR
axes[1].plot(t, np.abs(snr), linewidth=2, color="purple")
axes[1].axhline(8, color="red", linestyle="--", linewidth=2, label="Detection Threshold (SNR=8)")
axes[1].axvline(peak_time, color="red", linestyle="--", linewidth=2, alpha=0.5)
axes[1].scatter(
    [peak_time],
    [np.abs(peak_snr)],
    color="red",
    s=100,
    zorder=5,
    label=f"Peak SNR = {np.abs(peak_snr):.1f}",
)
axes[1].set_title("Matched Filter Signal-to-Noise Ratio", fontsize=12, fontweight="bold")
axes[1].set_xlabel("Time (s)")
axes[1].set_ylabel("SNR")
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim(0, max(15, np.abs(peak_snr) * 1.2))

plt.tight_layout()
plt.show()

## 6. Parameter Estimation

In [ ]:
# Estimate chirp mass from frequency evolution
# Extract region around merger
merger_window = (t > peak_time - 0.5) & (t < peak_time)
t_window = t[merger_window]
f_window = freq_evolution[merger_window]

# Fit frequency evolution to estimate chirp mass
# f(t) ~ (t_c - t)^(-3/8) for inspiral
from scipy.optimize import curve_fit


def f_inspiral(t, M_chirp_fit, t_coal_fit):
    tau = np.maximum(t_coal_fit - t, 1e-3)
    G = 6.67430e-11
    c = 299792458
    M_sun = 1.989e30
    return (
        (1 / (8 * np.pi))
        * (5 / (256 * tau)) ** (3 / 8)
        * (G * M_chirp_fit * M_sun / c**3) ** (-5 / 8)
    )


try:
    popt, _ = curve_fit(f_inspiral, t_window, f_window, p0=[M_chirp, peak_time], maxfev=5000)
    M_chirp_est, t_coal_est = popt

    print("\nParameter Estimation:")
    print(f"  True chirp mass: {M_chirp:.2f} M☉")
    print(f"  Estimated chirp mass: {M_chirp_est:.2f} M☉")
    print(f"  Error: {abs(M_chirp_est - M_chirp) / M_chirp * 100:.1f}%")
    print("\n  True coalescence time: 2.000 s")
    print(f"  Estimated coalescence time: {t_coal_est:.3f} s")
except:
    print("\nParameter estimation: Using simplified approach")
    print(f"  Chirp mass: ~{M_chirp:.1f} M☉ (from simulation)")
    print(f"  Coalescence time: ~{peak_time:.3f} s (from matched filter)")

## 7. Q-Transform (Time-Frequency)

In [ ]:
# Enhanced time-frequency representation
# Use spectrogram with better resolution
f_q, t_q, Sxx_q = signal.spectrogram(data, sample_rate, nperseg=256, noverlap=250)

fig, ax = plt.subplots(figsize=(14, 6))
im = ax.pcolormesh(
    t_q, f_q, 10 * np.log10(Sxx_q + 1e-30), shading="gouraud", cmap="viridis", vmin=-10, vmax=10
)
ax.set_ylim(20, 400)
ax.set_xlim(peak_time - 1, peak_time + 0.5)
ax.axvline(peak_time, color="red", linestyle="--", linewidth=2, label="Coalescence Time")
ax.set_title("Q-Transform: Chirp Signal Visible", fontsize=14, fontweight="bold")
ax.set_xlabel("Time (s)", fontsize=12)
ax.set_ylabel("Frequency (Hz)", fontsize=12)
plt.colorbar(im, ax=ax, label="Power (dB)")
ax.legend(loc="upper left")

plt.tight_layout()
plt.show()

print("\nThe chirp is the upward sweep in frequency!")

## 8. Summary Statistics

In [ ]:
# Summary
summary = pd.DataFrame(
    {
        "Parameter": [
            "Mass 1",
            "Mass 2",
            "Total Mass",
            "Chirp Mass",
            "Distance",
            "Peak Strain",
            "RMS Noise",
            "Matched Filter SNR",
            "Detection Status",
            "Coalescence Time",
        ],
        "Value": [
            f"{m1} M☉",
            f"{m2} M☉",
            f"{M_total} M☉",
            f"{M_chirp:.1f} M☉",
            f"{distance} Mpc",
            f"{np.max(np.abs(h_signal)):.2e}",
            f"{np.std(noise):.2e}",
            f"{np.abs(peak_snr):.1f}",
            "DETECTED" if np.abs(peak_snr) > 8 else "Not Detected",
            f"{peak_time:.3f} s",
        ],
    }
)

print("=" * 70)
print("GRAVITATIONAL WAVE DETECTION SUMMARY")
print("=" * 70)
print(summary.to_string(index=False))
print("=" * 70)

# Save
summary.to_csv("gw_detection_summary.csv", index=False)
print("\n✓ Summary saved to gw_detection_summary.csv")

## Key Takeaways

### Detection Method
- **Matched filtering** is the optimal detection method for signals with known waveform shape
- Detection threshold: SNR > 8 (5σ significance)
- Real LIGO uses advanced matched filtering with thousands of templates

### Signal Characteristics
- **Chirp**: Frequency and amplitude increase as black holes spiral inward
- **Inspiral-Merger-Ringdown**: Three distinct phases
- Strain amplitude: ~10⁻²¹ (incredibly small!)

### Parameter Estimation
- Chirp mass: Most accurately measured parameter
- Individual masses: Require both detectors (H1, L1)
- Sky localization: Requires detector network

### Real LIGO Analysis
- Multiple templates spanning parameter space
- Bayesian parameter estimation
- Detector characterization and data quality
- False alarm rate < 1 per 10⁶ years

## Next Steps

1. **Access real LIGO data**: [GWOSC](https://www.gw-openscience.org/)
2. **Use PyCBC**: Production-grade GW analysis
3. **Parameter estimation**: MCMC/nested sampling
4. **Multi-detector analysis**: Combine H1, L1, V1
5. **Advanced waveforms**: Numerical relativity, IMR models

## Resources

- [GWOSC Tutorials](https://www.gw-openscience.org/tutorials/)
- [PyCBC Documentation](https://pycbc.org/)
- [GWpy Documentation](https://gwpy.github.io/)
- [LIGO Scientific Collaboration](https://www.ligo.org/)